# Bonus point
In this notebook we will build the unweighted and undirected communication graph among the different email senders and recipients using the NetworkX library.

The notebook is divided into 239402 parts:
- Creation of the proper database
- Building of the graph
- Getting all the communities in the graph and getting the main topics in each community 

# Creation of the proper database
In this phase we will create the proper 

### Import and read files

In [1]:
import pandas as pd
import numpy as np
import pycountry
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from nltk import wordpunct_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import defaultdict
import string
import community
import networkx as nx
import matplotlib.pyplot as plt
from gensim import corpora, models

/home/stefano/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
emails = pd.read_csv('hillary-clinton-emails/sentimentEmails.csv')
emails=emails[['ProcessedData']]
emails.head(2)

,ProcessedData
0,h : latest : syria aid qaddafi ... sid b6 thur...
1,chris stevens thx


In [3]:
fullDF = pd.read_csv('hillary-clinton-emails/Emails.csv')
fullDF.head(2)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...


In [4]:
fullDF.shape

(7945, 22)

In [5]:
emails.shape

(6456, 1)

### Merge the 2 databases

In [6]:
bodyDF=fullDF[['ExtractedBodyText','MetadataSubject']]

In [7]:
reducedDF=fullDF[np.sum(bodyDF.isnull(),axis=1)==0]
reducedDF.shape

(6456, 22)

In [8]:
reducedDF=reducedDF.reset_index()
del reducedDF['index']
reducedDF.head(3)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
1,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
2,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [9]:
emails.head(3)

,ProcessedData
0,h : latest : syria aid qaddafi ... sid b6 thur...
1,chris stevens thx
2,h : latest : syria aid qaddafi ... sid h < hro...


In [10]:
reducedDF['ProcessedData']=emails.ProcessedData
reducedDF.head(3)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText,ProcessedData
0,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...,h : latest : syria aid qaddafi ... sid b6 thur...
1,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...,chris stevens thx
2,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,h : latest : syria aid qaddafi ... sid h < hro...


### Process the reducedDF to get the finalDF

In [20]:
finalDF=reducedDF[['Id','SenderPersonId','ProcessedData']]
finalDF=finalDF.dropna()
finalDF=finalDF.set_index(finalDF.Id)
del finalDF['Id']
finalDF.SenderPersonId=finalDF.SenderPersonId.apply(lambda x: int(x))
finalDF.head()

,SenderPersonId,ProcessedData
Id,,
3,32,chris stevens thx
5,80,h : latest : syria aid qaddafi ... sid h < hro...
6,80,right - wing extremist behind anti - muslim fi...
8,80,h : latest : syria aid qaddafi ... sid h < hro...
9,87,secretary ' remark fyi


In [21]:
finalDF.shape

(6433, 2)

# Reading EmailReceivers

In [ ]:
email_receivers=pd.read_csv('hillary-clinton-emails/EmailReceivers.csv')
email_receivers=email_receivers.set_index(email_receivers.Id)
del email_receivers['Id']

In [ ]:
len(set(email_receivers.PersonId))

In [ ]:
max(email_receivers.EmailId)

In [ ]:
def add_sender(x):
    try:
        x['Sender']=finalDF.loc[x.EmailId.iloc[0]].SenderPersonId
    except:
        x['Sender']=float('nan')
    return x

In [ ]:
email_receivers=email_receivers.groupby(email_receivers.EmailId).apply(add_sender)

In [ ]:
email_receivers[email_receivers.EmailId==6536]

In [ ]:
email_receivers.head()

In [ ]:
email_receivers=email_receivers.dropna()
email_receivers.Sender=email_receivers.Sender.apply(lambda x: int(x))
email_receivers.head()

In [ ]:
len(email_receivers.Sender.value_counts())

# Trying the libraries

In [ ]:
graph=nx.Graph()
for a, b in email_receivers[['Sender','PersonId']].itertuples(index=False):
    graph.add_edge(a,b)

In [ ]:
partition=community.best_partition(graph)

In [ ]:
set(partition.values())

In [ ]:
#drawing
size = float(len(set(partition.values())))
pos = nx.spring_layout(graph)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(graph, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(graph,pos, alpha=0.5)
plt.show()

# Add in finalDF a column with the community of the sender

In [ ]:
def add_community(x):
    try:
        x['Community']=int(partition[x.SenderPersonId.iloc[0]])
    except:
        x['Community']=float('nan')
    return x

In [ ]:
finalDF.head(2)

In [ ]:
finalDF=finalDF.groupby(finalDF.SenderPersonId).apply(add_community)
finalDF=finalDF.dropna()

In [ ]:
finalDF.Community=finalDF.Community.apply(lambda x: int(x))

In [ ]:
finalDF.head(4)

In [ ]:
finalDF.Community.value_counts()

In [ ]:
categories_list=list(set(partition.values()))

In [ ]:
categories_list

In [ ]:
trivial_words = ['u','f','w','h','j','us','fyi','would','fw','get',"--","—"]

In [ ]:
for cat in categories_list:
    df_community=finalDF[finalDF.Community==cat]
    text_string=' '.join(list(df_community.ProcessedData)).split(' ')
    text_string=[x for x in text_string if x[0] not in string.punctuation and not x.isdigit() and x not in trivial_words]
    
    total_dictionary={}
    for s in text_string:
        try:
            total_dictionary[s]+=1
        except:
            total_dictionary[s]=1
    
    words=[]
    cont=0
    while cont<20:
        word=max(total_dictionary, key=lambda k: total_dictionary[k])
        del total_dictionary[word]
        words.append(word)
        cont+=1
    print('Community',cat,':',' '.join(words))
    print()